In [1]:
%load_ext autoreload
%autoreload 2

# Dependencies

In [2]:
!uv pip install dotenv "numpy<2.0.0"

Using Python 3.11.13 environment at: /usr
Resolved 3 packages in 192ms
⠙ Preparing packages... (0/3)
⠙ Preparing packages... (0/3)
python-dotenv ------------------------------     0 B/19.78 KiB
⠙ Preparing packages... (0/3)
python-dotenv ------------------------------     0 B/19.78 KiB
⠙ Preparing packages... (0/3)
python-dotenv ------------------------------ 14.87 KiB/19.78 KiB
⠙ Preparing packages... (0/3)
python-dotenv ------------------------------ 14.87 KiB/19.78 KiB
⠙ Preparing packages... (0/3)
python-dotenv ------------------------------ 14.87 KiB/19.78 KiB
⠙ Preparing packages... (0/3)
python-dotenv ------------------------------ 14.87 KiB/19.78 KiB
⠙ Preparing packages... (0/3)
python-dotenv ------------------------------ 14.87 KiB/19.78 KiB
⠙ Preparing packages... (0/3)
python-dotenv ------------------------------ 19.78 KiB/19.78 KiB
⠙ Preparing packages... (0/3)
⠙ Preparing packages... (0/3)
⠙ Preparing packages... (0/3)
⠙ Preparing packages... (0/3)
⠙ Preparing packages...

In [3]:
!uv pip install openpipe-art openpipe --prerelease allow --no-cache-dir

[autoreload of numpy failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
    ^^^^^^^^^^^^^
TypeError: __class__ assignment: 'errstate' object layout differs from 'errstate'
]
[autoreload of numpy.ma.core failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/extension

Streaming output truncated to the last 5000 lines.
litellm    ------------------------------ 286.88 KiB/7.62 MiB
⠦ Preparing packages... (36/87)
opentelemetry-semantic-conventions-ai ------------------------------ 5.49 KiB/5.49 KiB
opentelemetry-exporter-otlp ------------------------------ 6.84 KiB/6.84 KiB
deprecated ------------------------------ 9.76 KiB/9.76 KiB
panza      ------------------------------ 10.37 KiB/10.37 KiB
fastapi-cli ------------------------------ 10.45 KiB/10.45 KiB
partial-json-parser ------------------------------ 10.63 KiB/10.63 KiB
python-json-logger ------------------------------ 15.08 KiB/15.08 KiB
anthropic  ------------------------------ 142.91 KiB/237.66 KiB
dnspython  ------------------------------ 302.40 KiB/306.28 KiB
openpipe   ------------------------------ 93.94 KiB/430.56 KiB
datasets   ------------------------------ 460.12 KiB/480.03 KiB
docutils   ------------------------------ 300.78 KiB/535.33 KiB
openai     ------------------------------ 238.

# Environment variables

In [1]:
import os

# Optional
WANDB_API_KEY = ""
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY

# Optional
OPENPIPE_API_KEY = ""
if OPENPIPE_API_KEY:
    os.environ["OPENPIPE_API_KEY"] = OPENPIPE_API_KEY

MODEL_NAME = "001"
PROJECT = "generate-fhir-single-turn"
BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"
LEARNING_RATE = 1.2e-5
GROUPS_PER_STEP = 1
EVAL_STEPS = 50
VAL_SET_SIZE = 100
NUM_EPOCHS = 1
NUM_GENERATIONS = 6

# Main loop

In [ ]:
import datasets
import art
from art.local import LocalBackend
from openpipe import AsyncOpenPipe

backend = LocalBackend()
model = art.TrainableModel(
    name=MODEL_NAME,
    project=PROJECT,
    base_model=BASE_MODEL,
    _internal_config=art.dev.InternalModelConfig(
        init_args=art.dev.InitArgs(
            gpu_memory_utilization=0.75,
        ),
        peft_args=art.dev.PeftArgs(
            lora_alpha=8,
        ),
        trainer_args=art.dev.TrainerArgs(
            max_grad_norm=0.1,
        ),
    ),
)
await model.register(backend)
op_client = AsyncOpenPipe(api_key=os.getenv("OPENPIPE_API_KEY"))

## Load the training data
print("Loading training data...")
train_dataset: datasets.Dataset = datasets.load_dataset("../data/fhir-single-turn")

train_data_list: List[Dict[str, Any]] = list(train_dataset)  # type: ignore
print(f"Training data size: {len(train_data_list)}")

# Get OpenAI Client for the ART Model
openai_client = model.openai_client()

# Training Loop
start_step = await model.get_step()
print(f"Starting training from global step {start_step}")

data_iterator = art.utils.iterate_dataset(
    dataset=train_data_list,
    groups_per_step=GROUPS_PER_STEP,
    num_epochs=NUM_EPOCHS,
    initial_step=start_step,
    use_tqdm=True,
)

for batch_inputs, epoch, global_step, epoch_step in data_iterator:
  print(batch_inputs)
  print(epoch)
  print(global_step)
  print(epoch_step)
  break

INFO 06-06 12:38:21 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-06 12:38:22 [__init__.py:239] Automatically detected platform cuda.
